In [1]:
import pandas as pd
import numpy as np
import re

In [ ]:
#df_scrape_researchers = pd.read_excel('scrape_results_5.0.xlsm')
#df_scrape_dissertations = pd.read_excel('scrape_results_5.0.xlsm', sheet_name = 'Dissertations')

In [2]:
df_VR = pd.read_excel('VR-ansökningar 2012-2016.xlsx', sheet_name = 'data 12-16')
df_VR.drop(df_VR.columns[22:],axis=1, inplace=True)

df_VR['Dnr'] = df_VR['Dnr'].astype(str)

In [7]:
df_RA = pd.read_excel('RA_Sheet_IL.xlsx')

df_RA = df_RA[(df_RA['count']>2)] #Only keep observations that have at least 3 Dnrs
df_RA.reset_index(drop=True, inplace=True) #reset index
df_RA['year'] = df_RA['year'].astype(str) #make year string for all observations
df_RA['year_len'] = df_RA['year'].str.len() #column of length of year –> helps us determine which observations have more than one year reported. 
df_RA.replace('nan', np.nan, inplace=True)

df_RA['COMPLETE'] = False #Make COMPLETE into a true/false column - true if all is complete

In [8]:
df_RA

,ID,COMPLETE,name,surname,fullname,app_ids,count,personnummer_cv,year,years_unsure,birth_year,in_sweden,institution,year_len
0,2,False,Maria,Swanberg,Maria Swanberg,"['2016-00992', '2013-03310', '2012-02563', '20...",6,19770510-0241,2007,NaN,1977.0,1,Karolinska Institutet,4
1,16,False,Gianni,Garcia-Faroldi,Gianni Garcia-Faroldi,"['2016-01158', '2014-02177', '2015-03377']",3,NaN,NaN,NaN,NaN,0,NaN,3
2,17,False,Malin,Jonsson Fagerlund,Malin Jonsson Fagerlund,"['2016-01172', '2012-01982', '2013-02620', '20...",5,19740908-8585,2006,"2006, 2015",NaN,1,Karolinska Institutet,4
3,18,False,Ulrika,Islander,Ulrika Islander,"['2016-01192', '2012-01561', '2013-02839', '20...",4,19750420-4988,2007,NaN,1975.0,1,Göteborgs universitet,4
4,23,False,Magnus,Sandberg,Magnus Sandberg,"['2016-01211', '2016-02405', '2015-03320']",3,NaN,"2013, 2010","2013, 2010",NaN,1,"Karolinska Institutet, Lunds universitet",10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3527,13005,False,Jens,Frauenfeld,Jens Frauenfeld,"['2013-05734', '2014-05589', '2015-04445']",3,NaN,NaN,NaN,NaN,0,NaN,3
3528,13008,False,Sandra,Eriksson,Sandra Eriksson,"['2013-05745', '2014-05634', '2015-03811']",3,"['19790218-1648', '19560211-1634']",2008,NaN,1979.0,1,Uppsala universitet,4
3529,13027,False,Ulrich,Weininger,Ulrich Weininger,"['2013-05953', '2014-04320', '2015-04889']",3,19780506-2879,2009,NaN,NaN,0,NaN,4
3530,13040,False,Anna,Bjällmark,Anna Bjällmark,"['2013-06039', '2014-02297', '2015-04183']",3,NaN,2009,NaN,1980.0,1,Kungliga Tekniska högskolan,4


## Idea: For people with in_sweden=1, institution, one year, no years unsure: 

#### Indications that it is the same person: 
#### 1: Repeated SCB-codes in all applications (same topics)
#### 2: Repeated keywords 
#### 3: Repeated projecttitles (need to remove common words: is, of, and, by, are, am, the
#### 4: Overlapping institutions - same institution
#### 5: Check those that do not fulfil any of these criteria manually

In [9]:
names = []
for k in df_RA.index:
    if df_RA['in_sweden'][k] == 1 and pd.isna(df_RA['years_unsure'][k])==True and df_RA['year_len'][k]==4 and pd.isna(df_RA['institution'][k])==False and pd.isna(df_RA['birth_year'][k])==False:
        print(df_RA['fullname'][k])  #all observations in sweden, with only one year reported, no years unsure, institution and birth year included
        names.append(df_RA['fullname'][k])

len(names) #List of all people with more than two applications, most info already there. 

Maria Swanberg
Ulrika Islander
Martin Eriksson Crommert
Anne-Lie Blomström
Johan Öckinger
Ulrica von Thiele Schwarz
Ninib Baryawno
Joakim Esbjörnsson
Eva Angenete
Caroline Palm Apergi
Mattias Forsell
Dariush Mokhtari
Rolf Lood
Gianluca Maddalo
Johan Zelano
Michael Hultström
Katarina Howner
Celine Fernandez
Sebastian Lundström
Ida Nilsson
Sara Gredmark Russ
Nitya Jayaram-Lindström
Karin Jensen
Camilla Pramfalk
Eric Westman
Sofie Singbrant Söderberg
Isabella Karlsson
Håkan Littbrand
Henrik Seth
Per Hydbring
Maja Malmberg
Emma Ahlqvist
Wibke Jonas
Misha Zilberter
Johan Ursing
My Andersson
Johanna Lindahl
Malin Werling
Zongli Zheng
Xia Shen
Linus Olson
Kim Kultima
Helena Nordenstedt
Andrei Malinovschi
Ulf Ribacke
Ingegerd Elvers
Boel Brynedal
Jessica Cedervall
Sofia Waldemarson
Badrul Hasan
Johan Öinert
Karl Börjesson
Isaac Skog
Samuel Genheden
Stefan Engblom
Martin Berggren
Sofia Mayans
Anders Enjin
Emma Aneheim
Petter Bivall
Maria Hahlin
Martin Sahlberg
Erik Holmqvist
Abhilash Sugunan
Jo

1969

### To check an individual (example): 

In [10]:
n = 'Shafiq Rehman'

filt = (df_RA['fullname']==n) 
Dnrs = str(df_RA[filt]['app_ids'].values) #extract dnr-numbers for each name
Dnrs = re.findall('\'(.+?)\'', Dnrs) #So that it becomes a list of string variables (correct format)
SCB = list(df_VR[df_VR['Dnr'].isin(Dnrs)]['Alla SCBkoder'].values) 

for i in range(len(SCB)): #loop through the list of SCB codes (from all applications for one person)
    SCB[i] = str(SCB[i]) #make sure that all are in string format
    SCB[i] = SCB[i].replace(' ,', '')
    SCB[i] = SCB[i].split(', ')
    
common_SCB = set.intersection(*[set(list) for list in SCB]) #set of intersecting SCB-numbers from each application
if len(common_SCB) > 0:
    print(n + ' is complete')
else: 
    print(n + ' needs to be checked')    
    
filt = (df_RA['fullname']==n) 
Dnrs = str(df_RA[filt]['app_ids'].values)
Dnrs = re.findall('\'(.+?)\'', Dnrs)
df_VR[df_VR['Dnr'].isin(Dnrs)]

Shafiq Rehman needs to be checked


,Dnr,Ärendeår,Efternamn,Förnamn,Kön,Medelsförvaltare,Beslut,Klustrad bidragsform,Bidragsform,Inriktning,...,Beslutsorgan (ÄRK),Ämnesområde,HuvudBG,Alla SCBkoder,TotalBevBelopp,Antal Beviljade År,ProjekttitelSv,ProjekttitelEn,Nyckelord,Unnamed: 21
24964,2013-05202,2013,Rehman,Shafiq,Man,Umeå universitet,Avslag,Etablering - Unga forskare,Projektbidrag Unga forskare,Fri,...,Ämnesrådet för naturvetenskap och teknikvetenskap,NT,NT-2,"10204, 10206, 21102",0.0,0,Haptisk Broadcasting av ´känslor´ från multimedia,Touch of Sensation: Haptic Broadcasting of ´Em...,vibrotactile; mutimedia; facial expression; em...,NaN
25954,2014-05091,2014,Rehman,Shafiq,Man,Umeå universitet,Avslag,Etablering - Unga forskare,Projektbidrag Unga forskare,Fri,...,Ämnesrådet för naturvetenskap och teknikvetenskap,NT,NT-2,"10206, 10204, 21102",NaN,0,Haptisk Broadcasting av ´känslor´ från multimedia,Touch of Sensation: Haptic Broadcasting of ´Em...,emotions; mutimedia; TV-broadcasting; Haptics;...,NaN
26599,2015-03729,2015,ur Rehman,Shafiq,Man,Umeå universitet,Avslag,Etablering - Unga forskare,Projektbidrag Unga forskare,Fri,...,Ämnesrådet för naturvetenskap och teknikvetenskap,NT,NT-2,"20205, 20206, 21103",0.0,0,Broadcasting av ´känslor´ från multimedia,Touch of sensation: Haptic Broadcasting of ´Em...,"multimodal signal processing, Affective comput...",NaN


### Overlapping SCB-codes

In [13]:
names = ['Martin Andersson']
for n in names: #for each name in the list of names that fulfill certain criteria
    filt = (df_RA['fullname']==n) 
    Dnrs = str(df_RA[filt]['app_ids'].values) #extract dnr-numbers for each name
    Dnrs = re.findall('\'(.+?)\'', Dnrs) #So that it becomes a list of string variables (correct format)
    SCB = list(df_VR[df_VR['Dnr'].isin(Dnrs)]['Alla SCBkoder'].values) #Extract SCB codes from df_VR for each dnr-number
    print(SCB)
    for i in range(len(SCB)): #loop through the list of SCB codes (from all applications for one person)
        SCB[i] = str(SCB[i]) #make sure that all are in string format
        SCB[i] = SCB[i].replace(' ,', '')
        SCB[i] = SCB[i].split(', ') #creates list of dnr numbers for each application (from one person) inside one list
    print('\n', SCB)

    common_SCB = set.intersection(*[set(list) for list in SCB]) #set of intersecting SCB-numbers from each application
    print('\n', common_SCB)
    if len(common_SCB) > 0:
        df_RA.loc[filt, 'COMPLETE'] = True

['10606, 10609, 10615', '30109, 30108 ,', '10606, 10609, 10615', 30401, '20304, 20599 ,', '10403, 10402, 21001', '21001, 20502, 10403', '50203, 50201 ,', '20602, 20502, 21001', 50203, '10499, 10403, 10601', '50202, 50201 ,', '20304, 20306, 20599', '20304, 20306, 20599', '20499, 10404, 20304', '10602, 10611, 10615']

 [['10606', '10609', '10615'], ['30109', '30108'], ['10606', '10609', '10615'], ['30401'], ['20304', '20599'], ['10403', '10402', '21001'], ['21001', '20502', '10403'], ['50203', '50201'], ['20602', '20502', '21001'], ['50203'], ['10499', '10403', '10601'], ['50202', '50201'], ['20304', '20306', '20599'], ['20304', '20306', '20599'], ['20499', '10404', '20304'], ['10602', '10611', '10615']]

 set()


In [12]:
df_RA[(df_RA['COMPLETE']==False) & (df_RA['fullname'].isin(names)) ]

,ID,COMPLETE,name,surname,fullname,app_ids,count,personnummer_cv,year,years_unsure,birth_year,in_sweden,institution,year_len
6,28,False,Anne-Lie,Blomström,Anne-Lie Blomström,"['2016-01251', '2012-06555', '2016-05652']",3,19780503-2443,2010,NaN,1978.0,1,"Sveriges Lantbruksuniversitet, Sveriges lantbr...",4
12,45,False,Joakim,Esbjörnsson,Joakim Esbjörnsson,"['2016-01417', '2012-00261', '2012-06628']",3,19780128-4618,2010,NaN,1978.0,1,Lunds universitet,4
18,64,False,Gianluca,Maddalo,Gianluca Maddalo,"['2016-01649', '2012-00327', '2015-03489']",3,19791015-6657,2011,NaN,1979.0,1,Stockholms universitet,4
33,121,False,Karin,Jensen,Karin Jensen,"['2016-02269', '2015-01844', '2014-02891']",3,19791012-1982,2009,NaN,1979.0,1,Karolinska Institutet,4
37,130,False,Isabella,Karlsson,Isabella Karlsson,"['2016-02298', '2012-06711', '2013-00390', '20...",5,NaN,2011,NaN,1980.0,1,Göteborgs universitet,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3480,12592,False,Patrik,Nygren,Patrik Nygren,"['2012-05124', '2013-05913', '2014-04702', '20...",4,19791006-4851,2008,NaN,1979.0,1,Linköpings universitet,4
3482,12617,False,Nathalie,Scheers,Nathalie Scheers,"['2013-02323', '2014-03871', '2015-03205']",3,NaN,2011,NaN,1974.0,1,Chalmers tekniska högskola,4
3490,12679,False,Aisha Siddiqah,Ahmed,Aisha Siddiqah Ahmed,"['2013-03139', '2014-03543', '2015-03503']",3,19670530-0603,2010,NaN,1967.0,1,Karolinska Institutet,4
3521,12944,False,Shafiq,Rehman,Shafiq Rehman,"['2013-05202', '2014-05091', '2015-03729']",3,NaN,2010,NaN,1978.0,1,Umeå universitet,4


df_RA[(df_RA['COMPLETE']==False)&(df_RA['fullname'].isin(names))].head(50)

### Overlapping keywords

In [15]:
import re

for n in names: #for each name in the list of names that fulfill certain criteria
    filt = (df_RA['fullname']==n) 
    Dnrs = str(df_RA[filt]['app_ids'].values) #extract dnr-numbers for each name
    Dnrs = re.findall('\'(.+?)\'', Dnrs) #So that it becomes a list of string variables (correct format)
    keywords = list(df_VR[df_VR['Dnr'].isin(Dnrs)]['Nyckelord'].values) #Extract SCB codes from df_VR for each dnr-number
   # print(keywords)
    for i in range(len(keywords)): #loop through the list of keywords (from all applications for one person)
        keywords[i] = str(keywords[i]) #make sure that all are in string format
        keywords[i] = keywords[i].replace(' ,', '')
        keywords[i] = keywords[i].replace(';', ',')
        keywords[i] = keywords[i].lower()
        keywords[i] = keywords[i].split(', ') #creates list of dnr numbers for each application (from one person) inside one list
    #print(keywords)
    
    common_keywords = set.intersection(*[set(list) for list in keywords]) #set of intersecting SCB-numbers from each application
    if len(common_keywords) > 0:
        df_RA.loc[filt, 'COMPLETE'] = True

In [16]:
df_RA[(df_RA['COMPLETE']==False) & (df_RA['fullname'].isin(names)) ]

,ID,COMPLETE,name,surname,fullname,app_ids,count,personnummer_cv,year,years_unsure,birth_year,in_sweden,institution,year_len
6,28,False,Anne-Lie,Blomström,Anne-Lie Blomström,"['2016-01251', '2012-06555', '2016-05652']",3,19780503-2443,2010,NaN,1978.0,1,"Sveriges Lantbruksuniversitet, Sveriges lantbr...",4
12,45,False,Joakim,Esbjörnsson,Joakim Esbjörnsson,"['2016-01417', '2012-00261', '2012-06628']",3,19780128-4618,2010,NaN,1978.0,1,Lunds universitet,4
18,64,False,Gianluca,Maddalo,Gianluca Maddalo,"['2016-01649', '2012-00327', '2015-03489']",3,19791015-6657,2011,NaN,1979.0,1,Stockholms universitet,4
33,121,False,Karin,Jensen,Karin Jensen,"['2016-02269', '2015-01844', '2014-02891']",3,19791012-1982,2009,NaN,1979.0,1,Karolinska Institutet,4
37,130,False,Isabella,Karlsson,Isabella Karlsson,"['2016-02298', '2012-06711', '2013-00390', '20...",5,NaN,2011,NaN,1980.0,1,Göteborgs universitet,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3445,12435,False,Mohammadreza,Shariatgorji,Mohammadreza Shariatgorji,"['2012-04151', '2013-04588', '2014-02414', '20...",4,19740923-2811,2010,NaN,1974.0,1,Stockholms universitet,4
3480,12592,False,Patrik,Nygren,Patrik Nygren,"['2012-05124', '2013-05913', '2014-04702', '20...",4,19791006-4851,2008,NaN,1979.0,1,Linköpings universitet,4
3482,12617,False,Nathalie,Scheers,Nathalie Scheers,"['2013-02323', '2014-03871', '2015-03205']",3,NaN,2011,NaN,1974.0,1,Chalmers tekniska högskola,4
3521,12944,False,Shafiq,Rehman,Shafiq Rehman,"['2013-05202', '2014-05091', '2015-03729']",3,NaN,2010,NaN,1978.0,1,Umeå universitet,4


### Overlapping projecttitles

In [17]:
for n in names: #for each name in the list of names that fulfill certain criteria
    filt = (df_RA['fullname']==n) 
    Dnrs = str(df_RA[filt]['app_ids'].values) #extract dnr-numbers for each name
    Dnrs = re.findall('\'(.+?)\'', Dnrs) #So that it becomes a list of string variables (correct format)
    titlewords = list(df_VR[df_VR['Dnr'].isin(Dnrs)]['ProjekttitelEn'].values) #Extract SCB codes from df_VR for each dnr-number
   # print(keywords)
    to_remove = ['of', 'the', 'and', 'a', 'an', 'to', 'if', 'this', 'an']

    for i in range(len(titlewords)):
        titlewords[i] = str(titlewords[i])
        titlewords[i] = titlewords[i].lower()
        titlewords[i] = titlewords[i].split(' ')
        for word in list(titlewords[i]):
            if word in to_remove:
                titlewords[i].remove(word)
    
    common_titlewords = set.intersection(*[set(list) for list in titlewords]) #set of intersecting SCB-numbers from each application
    if len(common_titlewords) > 0:
        df_RA.loc[filt, 'COMPLETE'] = True

In [18]:
df_RA[(df_RA['COMPLETE']==False) & (df_RA['fullname'].isin(names))]

,ID,COMPLETE,name,surname,fullname,app_ids,count,personnummer_cv,year,years_unsure,birth_year,in_sweden,institution,year_len
6,28,False,Anne-Lie,Blomström,Anne-Lie Blomström,"['2016-01251', '2012-06555', '2016-05652']",3,19780503-2443,2010,NaN,1978.0,1,"Sveriges Lantbruksuniversitet, Sveriges lantbr...",4
18,64,False,Gianluca,Maddalo,Gianluca Maddalo,"['2016-01649', '2012-00327', '2015-03489']",3,19791015-6657,2011,NaN,1979.0,1,Stockholms universitet,4
33,121,False,Karin,Jensen,Karin Jensen,"['2016-02269', '2015-01844', '2014-02891']",3,19791012-1982,2009,NaN,1979.0,1,Karolinska Institutet,4
37,130,False,Isabella,Karlsson,Isabella Karlsson,"['2016-02298', '2012-06711', '2013-00390', '20...",5,NaN,2011,NaN,1980.0,1,Göteborgs universitet,4
40,139,False,Per,Hydbring,Per Hydbring,"['2016-02379', '2012-01966', '2014-03580', '20...",4,19800524-2436,2009,NaN,1980.0,1,Sveriges lantbruksuniversitet,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,12088,False,Anna,Rising,Anna Rising,"['2012-01590', '2013-03113', '2014-02408', '20...",4,NaN,2007,NaN,1976.0,1,"Sveriges Lantbruksuniversitet, Sveriges lantbr...",4
3411,12252,False,Petros,Souvatzis,Petros Souvatzis,"['2012-02662', '2013-03759', '2014-03786']",3,19700309-1415,2007,NaN,1970.0,1,Uppsala universitet,4
3441,12425,False,Emil,Egecioglu,Emil Egecioglu,"['2012-04083', '2014-03560', '2015-03388']",3,NaN,2007,NaN,1977.0,1,Göteborgs universitet,4
3445,12435,False,Mohammadreza,Shariatgorji,Mohammadreza Shariatgorji,"['2012-04151', '2013-04588', '2014-02414', '20...",4,19740923-2811,2010,NaN,1974.0,1,Stockholms universitet,4


### Same "medelsförvaltare"
#### Motivation: There are observations that are obviously the same individual, but keywords/SCB/projecttitles do not completely overlap. 
#### Small likelihood that there are several people with same name at same institution while criteria for being in list of names is fulfilled

In [19]:
for n in names: #for each name in the list of names that fulfill certain criteria
    filt = (df_RA['fullname']==n) 
    Dnrs = str(df_RA[filt]['app_ids'].values) #extract dnr-numbers for each name
    Dnrs = re.findall('\'(.+?)\'', Dnrs) #So that it becomes a list of string variables (correct format)
    mf = list(df_VR[df_VR['Dnr'].isin(Dnrs)]['Medelsförvaltare'].values) 

    for i in range(len(mf)): #loop through the list of SCB codes (from all applications for one person)
        mf[i] = str(mf[i]) #make sure that all are in string format
        mf[i] = mf[i].lower() #some have different uppercase/lowercase letters
    
    if mf.count(mf[0]) == len(mf):
        df_RA.loc[filt, 'COMPLETE'] = True

In [21]:
df_RA[(df_RA['COMPLETE']==False) & (df_RA['fullname'].isin(names))]

,ID,COMPLETE,name,surname,fullname,app_ids,count,personnummer_cv,year,years_unsure,birth_year,in_sweden,institution,year_len
18,64,False,Gianluca,Maddalo,Gianluca Maddalo,"['2016-01649', '2012-00327', '2015-03489']",3,19791015-6657,2011,NaN,1979.0,1,Stockholms universitet,4
37,130,False,Isabella,Karlsson,Isabella Karlsson,"['2016-02298', '2012-06711', '2013-00390', '20...",5,NaN,2011,NaN,1980.0,1,Göteborgs universitet,4
41,143,False,Maja,Malmberg,Maja Malmberg,"['2016-02454', '2013-00314', '2013-06531', '20...",7,19820516-1428,2013,NaN,1982.0,1,Karolinska Institutet,4
53,173,False,Xia,Shen,Xia Shen,"['2016-02832', '2013-07241', '2014-00371', '20...",5,NaN,2012,NaN,1985.0,1,Uppsala universitet,4
77,244,False,Martin,Berggren,Martin Berggren,"['2016-03417', '2012-03681', '2013-03706', '20...",7,"['19680630-4074', '19810501-5534', '19560802-8...",2009,NaN,1981.0,1,Umeå universitet,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3331,11508,False,Peter,Järver,Peter Järver,"['2016-04543', '2013-04455', '2014-05567']",3,NaN,2007,NaN,1978.0,1,Stockholms universitet,4
3332,11515,False,Dennis,Eriksson,Dennis Eriksson,"['2016-04591', '2013-04162', '2013-04592', '20...",5,NaN,2009,NaN,1976.0,1,Sveriges lantbruksuniversitet,4
3340,11551,False,Sinisa,Bjelic,Sinisa Bjelic,"['2016-04786', '2013-05687', '2014-04978', '20...",4,NaN,2007,NaN,1977.0,1,Uppsala universitet,4
3384,12088,False,Anna,Rising,Anna Rising,"['2012-01590', '2013-03113', '2014-02408', '20...",4,NaN,2007,NaN,1976.0,1,"Sveriges Lantbruksuniversitet, Sveriges lantbr...",4


In [23]:
#Exempel: Två personer med samma namn. 

df_VR[(df_VR['fullname']=='Martin Berggren')]['ProjekttitelEn'].values 

array(['The role of lake plankton respiration in the global carbon cycle',
       'Optimization of devices for wave transmission',
       'Acoustic design optimization methods for thin structures and for systems subject to a nonuniform background flow',
       'Terrestrial export of bioavailable nutrients and labile organic carbon: Effects on nutrient stoichiometry and plankton metabolism in boreal freshwater ecosystems',
       'Terrestrial export of bioavailable nutrients and labile organic carbon: effects on nutrient stoichiometry and plankton metabolism in boreal freshwater ecosystems',
       'An empirical framework for large-scale modeling of dissolved organic carbon fluxes across soils and water',
       'New view on aquatic ecosystem response to nutrient loading in a changing environment'],
      dtype=object)